In [7]:
import pandas as pd
import numpy as np
import os
import cv2
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.utils import to_categorical
from keras import layers
from keras import models          
from keras import optimizers
from keras import callbacks

In [8]:
directory = 'D:/PCV/project/dataset'

In [9]:
def categorized_from_directory(path):
    """Returns a Pandas dataframe with the `category` and `path` of each image."""
    rows = []
    for category in os.listdir(path):
        category_path = os.path.join(path, category)
        for image in os.listdir(category_path):
            image_path = os.path.join(category_path, image)
            rows.append({'category': category, 'path': image_path})
    return pd.DataFrame(rows)

In [10]:
df = categorized_from_directory(directory)
print(df.head())

  category                                               path
0      0x0  D:/PCV/project/dataset\0x0\2023-11-19-23100707...
1      0x0  D:/PCV/project/dataset\0x0\2023-11-19-23100714...
2      0x0  D:/PCV/project/dataset\0x0\2023-11-19-23100720...
3      0x0  D:/PCV/project/dataset\0x0\2023-11-19-23100727...
4      0x0  D:/PCV/project/dataset\0x0\2023-11-19-23100733...


In [11]:
img = cv2.imread('D:/PCV/project/data/output/0x0/20180505_182542.jpg')
img.shape

(100, 100, 3)

In [12]:
# Fungsi untuk menghitung magnitudo gradien pada gambar
def calculate_magnitude_gradient(image):
    # Terapkan filter Sobel
    gradien_x = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
    gradien_y = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)
    
    # Hitung magnitudo gradien
    magnitude = np.sqrt(gradien_x**2 + gradien_y**2)
    
    # Normalisasi
    magnitude = (magnitude / np.max(magnitude)) * 255
    magnitude = np.uint8(np.floor(magnitude))
    
    return magnitude

# Buat direktori output untuk menyimpan hasil magnitudo gradien
output_directory = 'D:/PCV/project/data/output'
os.makedirs(output_directory, exist_ok=True)

# Loop melalui gambar dalam dataset dan hitung magnitudo gradien
for index, row in df.iterrows():
    image_path = row['path']
    category = row['category']
    image = cv2.imread(image_path)
    
    # Hitung magnitudo gradien
    magnitude_image = calculate_magnitude_gradient(image)
    
    # Simpan hasil magnitudo gradien
    output_category_directory = os.path.join(output_directory, category)
    os.makedirs(output_category_directory, exist_ok=True)
    output_image_path = os.path.join(output_category_directory, os.path.basename(image_path))
    cv2.imwrite(output_image_path, magnitude_image)

print("Proses selesai.")


error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\deriv.cpp:419: error: (-215:Assertion failed) !_src.empty() in function 'cv::Sobel'


In [6]:
image_size = (100, 100)
data = []

for category in os.listdir(output_directory):
    category_directory = os.path.join(output_directory, category)
    for image_name in os.listdir(category_directory):
        image_path = os.path.join(category_directory, image_name)
        image = cv2.imread(image_path)
        image = cv2.resize(image, image_size)
        data.append({'image': image, 'category': category})

# Konversi data ke DataFrame
output_df = pd.DataFrame(data)

In [7]:
print(image.shape)

(100, 100, 3)


In [8]:
def rotate_image(image, angle):
    rows, cols = image.shape[:2]
    M = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
    return cv2.warpAffine(image, M, (cols, rows))

def brightness_adjustment(image, alpha=1.0, beta=0):
    return cv2.convertScaleAbs(image, alpha=alpha, beta=beta)

def zoom_image(image, scale_factor):
    height, width = image.shape[:2]
    zoom_matrix = np.float32([[scale_factor, 0, (1 - scale_factor) * width / 2],
                              [0, scale_factor, (1 - scale_factor) * height / 2]])
    return cv2.warpAffine(image, zoom_matrix, (width, height))

# Contoh penerapan augmentasi
rotated_image = rotate_image(image, 30)
brightened_image = brightness_adjustment(image, alpha=1.5, beta=50)
zoomed_image = zoom_image(image, 0.5)


In [9]:
print(image.shape)

(100, 100, 3)


In [10]:
# Membagi data menjadi data pelatihan dan pengujian
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(output_df['image'], output_df['category'], test_size=0.2, random_state=42)

In [12]:
num_categories = output_df['category'].nunique()
num_categories

28

In [13]:
print(y_train.shape)

(2240,)


In [14]:
image.shape

(100, 100, 3)

In [15]:

# Mengonversi gambar menjadi array dan menyesuaikannya
X_train = np.array(X_train.tolist()) / 255.0
X_test = np.array(X_test.tolist()) / 255.0

# Membuat model
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.3))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(num_categories, activation='softmax'))

# Mengkompilasi model
model.compile(optimizer=optimizers.RMSprop(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

# Melatih model
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

# Mengevaluasi model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test loss:", test_loss)
print("Test accuracy:", test_accuracy)


c:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\optimizers\legacy\rmsprop.py:143: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/100


ValueError: in user code:

    File "c:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\engine\training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\engine\training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\engine\training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\engine\training.py", line 1051, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "c:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\engine\training.py", line 1109, in compute_loss
        return self.compiled_loss(
    File "c:\Users\USER\AppDat

In [16]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',input_shape=(100, 100, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.3))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(num_categories, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

In [17]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 98, 98, 32)        896       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 49, 49, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 47, 47, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 23, 23, 64)       0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 21, 21, 128)       73856     
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 10, 10, 128)     

In [5]:
import cv2
import numpy as np

def preprocess_image(image):
    # Konversi gambar ke skala abu-abu
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Reduksi noise menggunakan GaussianBlur
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Deteksi tepi menggunakan Canny
    edges = cv2.Canny(blurred, 50, 150)

    return edges

def find_domino_contours(edges):
    # Temukan kontur dalam gambar tepi
    contours, _ = cv2.findContours(edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Pilih kontur dengan area tertentu (sesuaikan sesuai kebutuhan)
    selected_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > 1000]

    return selected_contours

def find_domino_pips(domino_contour, original_image):
    # Dapatkan batas kotak dari kontur domino
    x, y, w, h = cv2.boundingRect(domino_contour)

    # Potong bagian domino dari gambar asli
    domino_roi = original_image[y:y+h, x:x+w]

    # Konversi ke skala abu-abu
    gray_roi = cv2.cvtColor(domino_roi, cv2.COLOR_BGR2GRAY)

    # Reduksi noise menggunakan GaussianBlur
    blurred_roi = cv2.GaussianBlur(gray_roi, (5, 5), 0)

    # Deteksi tepi menggunakan Canny
    edges_roi = cv2.Canny(blurred_roi, 50, 150)

    # Deteksi lingkaran menggunakan Hough Circle Transform
    circles = cv2.HoughCircles(edges_roi, cv2.HOUGH_GRADIENT, dp=1, minDist=20, param1=50, param2=30, minRadius=10, maxRadius=30)

    if circles is not None:
        circles = np.uint16(np.around(circles))
        num_pips = len(circles[0, :])

        # Tampilkan jumlah bulatan/domino pips
        print("Jumlah bulatan:", num_pips)

        # Gambarkan lingkaran di sekitar setiap bulatan/domino pip
        for i in circles[0, :]:
            cv2.circle(domino_roi, (i[0], i[1]), i[2], (0, 255, 0), 2)
            cv2.circle(domino_roi, (i[0], i[1]), 2, (0, 0, 255), 3)

        # Tampilkan ROI untuk verifikasi visual
        cv2.imshow("Domino ROI", domino_roi)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    else:
        print("Tidak ada lingkaran (bulatan) yang terdeteksi.")

def main():
    # Buka kamera
    cap = cv2.VideoCapture(0)

    while True:
        # Ambil frame dari kamera
        ret, frame = cap.read()

        # Pra-pemrosesan gambar
        edges = preprocess_image(frame)

        # Temukan kontur domino
        domino_contours = find_domino_contours(edges)

        # Loop melalui setiap kontur domino
        for contour in domino_contours:
            # Temukan dan hitung bulatan/domino pips di dalam setiap kontur
            find_domino_pips(contour, frame)

        # Tampilkan frame asli dengan kotak kontur
        cv2.drawContours(frame, domino_contours, -1, (0, 255, 0), 2)
        cv2.imshow("Domino Detection", frame)

        # Keluar dari loop jika tombol 'q' ditekan
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Tutup kamera dan jendela OpenCV
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

In [122]:
import cv2

def preprocess_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, 50, 150)
    return edges, blurred

# def preprocess_image(image):
#     gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#     blurred = cv2.GaussianBlur(gray, (3, 3), 0)
    
#     # Filter Sobel untuk mendeteksi tepi
#     sobel_x = cv2.Sobel(blurred, cv2.CV_64F, 1, 0, ksize=1)
#     sobel_y = cv2.Sobel(blurred, cv2.CV_64F, 0, 1, ksize=1)
    
#     edges = cv2.bitwise_or(cv2.convertScaleAbs(sobel_x), cv2.convertScaleAbs(sobel_y))
    
#     return edges, blurred

# def preprocess_image(image):
#     gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#     blurred = cv2.GaussianBlur(gray, (3, 3), 0)
    
#     # Metode thresholding untuk deteksi tepi
#     _, edges = cv2.threshold(blurred, 100, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
#     return edges, blurred

def find_square_cards(contours):
    selected_cards = []
    for contour in contours:
        perimeter = cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, 0.04 * perimeter, True)
        
        if len(approx) == 4:
            x, y, w, h = cv2.boundingRect(contour)
            aspect_ratio = float(w) / h
            
            # Misalkan kartu memiliki perbandingan aspek mendekati 1
            # Sesuaikan dengan karakteristik kartu yang ingin dideteksi
            if 0.4 <= aspect_ratio <= 0.7:
                selected_cards.append(contour)
    return selected_cards


# def find_domino_contours(edges):
#     contours, _ = cv2.findContours(edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#     selected_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > 100]
#     return selected_contours

def main():
    cap = cv2.VideoCapture(1)

    while True:
        ret, frame = cap.read()

        live_frame = frame.copy()
        edges, preprocessed_frame = preprocess_image(frame)

        domino_contours = find_domino_contours(edges)

        for contour in domino_contours:
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(live_frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        cv2.imshow("Domino Detection", live_frame)
        cv2.imshow("Preprocessed Frame - Edges", edges)  # Show Canny edges
        #cv2.imshow("Preprocessed Frame - Blurred", preprocessed_frame)  # Show blurred image

        # Save the detected result if there are contours detected
        if len(domino_contours) > 0:
            cv2.imwrite("detected_card.jpg", live_frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


In [119]:
import cv2
import time
import os

# Fungsi preprocess_image() dan find_square_cards() tetap sama seperti sebelumnya

def CreateDataSet(sDirektoriData, sKelas, NoKamera, FrameRate):
    sDirektoriKelas = os.path.join(sDirektoriData, sKelas)
    if not os.path.exists(sDirektoriKelas):
        os.makedirs(sDirektoriKelas)
  
    cap = cv2.VideoCapture(NoKamera)
    TimeStart = 0
    Recording = False  # Variabel untuk menandai apakah sedang merekam atau tidak
    
    while cap.isOpened():
        success, image = cap.read()
    
        if not success:
            print("Ignoring empty camera frame.")
            continue
      
        cv2.imshow('Hands', cv2.flip(image, 1))
        
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):  # Tekan 'q' untuk keluar dari loop
            break
        elif key == ord('t'):  # Tekan 't' untuk memulai atau menghentikan merekam
            if not Recording:
                Recording = True
                TimeStart = time.time()
            else:
                Recording = False
      
        # Jika sedang merekam, lakukan proses menyimpan data
        if Recording:
            TimeNow = time.time() 
            if TimeNow - TimeStart > 1 / FrameRate:
                edges, _ = preprocess_image(image)  # Proses gambar yang diambil dari kamera
                contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                
                selected_contours = find_square_cards(contours)  # Memilih kontur yang sesuai
                
                # Simpan setiap gambar yang sesuai dengan kontur yang terdeteksi
                for contour in selected_contours:
                    x, y, w, h = cv2.boundingRect(contour)
                    sfFile = os.path.join(sDirektoriKelas, f"frame_{int(TimeNow * 1000)}_{x}_{y}.jpg")
                    cv2.imwrite(sfFile, image[y:y+h, x:x+w])  # Simpan gambar yang sesuai dengan kontur
                    
                TimeStart = TimeNow

    cap.release()
    cv2.destroyAllWindows()

# Fungsi main() tetap sama seperti sebelumnya

if __name__ == "__main__":
    # Pengaturan dataset
    sDirektoriData = "path_to_your_dataset_directory"
    sKelas = "class_name"
    NoKamera = 0  # Nomor kamera
    FrameRate = 5  # Tingkat frame per detik
    
    # Membuat dataset
    CreateDataSet(sDirektoriData, sKelas, NoKamera, FrameRate)


In [123]:
%history

import cv2

# def preprocess_image(image):
#     gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#     blurred = cv2.GaussianBlur(gray, (5, 5), 0)
#     edges = cv2.Canny(blurred, 50, 150)
#     return edges, blurred

# def preprocess_image(image):
#     gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#     blurred = cv2.GaussianBlur(gray, (3, 3), 0)
    
#     # Filter Sobel untuk mendeteksi tepi
#     sobel_x = cv2.Sobel(blurred, cv2.CV_64F, 1, 0, ksize=1)
#     sobel_y = cv2.Sobel(blurred, cv2.CV_64F, 0, 1, ksize=1)
    
#     edges = cv2.bitwise_or(cv2.convertScaleAbs(sobel_x), cv2.convertScaleAbs(sobel_y))
    
#     return edges, blurred

def preprocess_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (3, 3), 0)
    
    # Metode thresholding untuk deteksi tepi
    _, edges = cv2.threshold(blurred, 100, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    return edges, blurred


def find_domino_contours(edges):
    contours, _ = cv

In [140]:
%recall 111

In [9]:
import cv2

# def preprocess_image(image):
#     gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#     blurred = cv2.GaussianBlur(gray, (5, 5), 0)
#     edges = cv2.Canny(blurred, 50, 150)
#     return edges, blurred

def preprocess_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (3, 3), 0)
    
    # Filter Sobel untuk mendeteksi tepi
    sobel_x = cv2.Sobel(blurred, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(blurred, cv2.CV_64F, 0, 1, ksize=3)
    
    edge = cv2.bitwise_or(cv2.convertScaleAbs(sobel_x), cv2.convertScaleAbs(sobel_y))

    # edges = cv2.Canny(edge, 50, 150)
        
    _, edges = cv2.threshold(edge, 100, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    return edges, blurred

# def preprocess_image(image):
#     gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#     blurred = cv2.GaussianBlur(gray, (3, 3), 0)
    
#     # Metode thresholding untuk deteksi tepi
#     _, edges = cv2.threshold(blurred, 100, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
#     return edges, blurred

def main():
    cap = cv2.VideoCapture(1)

    while True:
        ret, frame = cap.read()

        live_frame = frame.copy()
        edges, preprocessed_frame = preprocess_image(frame)

        contours, _ = cv2.findContours(edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        for contour in contours:
            perimeter = cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, 0.04 * perimeter, True)
            
            if len(approx) == 4:
                x, y, w, h = cv2.boundingRect(contour)
                aspect_ratio = float(w) / h
                aspect_ratio_a = float(h) / w
                
                # Misalkan kartu memiliki perbandingan aspek mendekati 1
                # Sesuaikan dengan karakteristik kartu yang ingin dideteksi
                if 0.3 <= aspect_ratio <= 0.8 or 0.3 <= aspect_ratio_a <= 0.8 :
                    cv2.rectangle(live_frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

                # if 0.3 <= aspect_ratio_a <= 0.8:
                #     cv2.rectangle(live_frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        cv2.imshow("Deteksi Kartu Kotak", live_frame)
        cv2.imshow("Frame Hasil Pra-Pemrosesan - Tepi", edges)  # Tampilkan tepi Canny

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

In [11]:
import cv2
import time
import os

def preprocess_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (3, 3), 0)
    
    # Filter Sobel untuk mendeteksi tepi
    sobel_x = cv2.Sobel(blurred, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(blurred, cv2.CV_64F, 0, 1, ksize=3)
    
    edge = cv2.bitwise_or(cv2.convertScaleAbs(sobel_x), cv2.convertScaleAbs(sobel_y))

    # edges = cv2.Canny(edge, 50, 150)
        
    _, edges = cv2.threshold(edge, 100, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    return edges, blurred

def detect_contours(edges, image):
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    detected_image = image.copy()
    detected_x, detected_y = 0, 0  # Inisialisasi nilai x dan y
    cropped_images = []

    for contour in contours:
        perimeter = cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, 0.04 * perimeter, True)

        if len(approx) == 4:
            x, y, w, h = cv2.boundingRect(contour)
            aspect_ratio = float(w) / h
            aspect_ratio_a = float(h) / w

            if 0.3 <= aspect_ratio <= 0.8 or 0.3 <= aspect_ratio_a <= 0.8:
                cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
                detected_x, detected_y = x, y  # Update nilai x dan y

                # Crop gambar di dalam kontur yang terdeteksi
                cropped_image = edges[y:y+h, x:x+w]
                cropped_images.append(cropped_image)

    return edges, detected_image, cropped_images, detected_x, detected_y

def CreateDataSet(sDirektoriData, sKelas, NoKamera, FrameRate, recording_duration):
    sDirektoriKelas = os.path.join(sDirektoriData, sKelas)
    if not os.path.exists(sDirektoriKelas):
        os.makedirs(sDirektoriKelas)

    cap = cv2.VideoCapture(NoKamera)
    TimeStart = 0
    Recording = False
    recording_end_time = 0
    x, y = 0, 0  # Inisialisasi nilai x dan y

    while cap.isOpened():
        success, image = cap.read()

        if not success:
            print("Ignoring empty camera frame.")
            continue

        cv2.imshow('Original', image)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
        elif key == ord('t'):
            if not Recording:
                edges, detected_image, cropped_images, x, y = detect_contours(*preprocess_image(image))
                cv2.imshow('Filtered Edges', edges)
                cv2.imshow('Detected Contours', detected_image)

                Recording = True
                TimeStart = time.time()
                recording_end_time = TimeStart + recording_duration  # Waktu akhir pencatatan

        if Recording:
            TimeNow = time.time()
            if TimeNow - TimeStart > 1 / FrameRate and TimeNow < recording_end_time:
                edges, _ = preprocess_image(image)

                # sfFile = os.path.join(sDirektoriKelas, f"edges_{int(TimeNow * 1000)}_{x}_{y}.jpg")
                # cv2.imwrite(sfFile, edges)

                cv2.imshow('Filtered Edges', edges)

                TimeStart = TimeNow

                # Simpan gambar yang di-crop dari kontur yang terdeteksi sebagai dataset
                for i, cropped_image in enumerate(cropped_images):
                    sfFile = os.path.join(sDirektoriKelas, f"cropped_{int(TimeNow * 1000)}_{x}_{y}_{i}.jpg")
                    cv2.imwrite(sfFile, cropped_image)  # Simpan gambar yang di-crop

                    cv2.imshow(f'Cropped Image {i}', cropped_image)

            # Mematikan kamera setelah mencapai waktu akhir pencatatan
            if TimeNow >= recording_end_time:
                print("Recording duration reached. Stopping the camera.")
                break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    sDirektoriData = "D:/PCV/project/dataset"
    sKelas = "1x1"
    NoKamera = 1  # Menggunakan kamera 1, sesuaikan jika menggunakan kamera lain
    FrameRate = 7
    recording_duration = 7  # Waktu pencatatan dalam detik

    CreateDataSet(sDirektoriData, sKelas, NoKamera, FrameRate, recording_duration)


In [1]:
import cv2
import time
import os

def preprocess_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (3, 3), 0)
    
    # Filter Sobel untuk mendeteksi tepi
    sobel_x = cv2.Sobel(blurred, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(blurred, cv2.CV_64F, 0, 1, ksize=3)
    
    edge = cv2.bitwise_or(cv2.convertScaleAbs(sobel_x), cv2.convertScaleAbs(sobel_y))

    # edges = cv2.Canny(edge, 50, 150)
        
    _, edges = cv2.threshold(edge, 100, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    return edges, blurred

def detect_contours(edges, image):
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    detected_image = image.copy()
    detected_x, detected_y = 0, 0  # Inisialisasi nilai x dan y
    cropped_images = []

    for contour in contours:
        perimeter = cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, 0.04 * perimeter, True)

        if len(approx) == 4:
            x, y, w, h = cv2.boundingRect(contour)
            aspect_ratio = float(w) / h
            aspect_ratio_a = float(h) / w

            if 0.3 <= aspect_ratio <= 0.8 or 0.3 <= aspect_ratio_a <= 0.8:
                cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
                detected_x, detected_y = x, y  # Update nilai x dan y

                # Crop gambar di dalam kontur yang terdeteksi
                cropped_image = edges[y:y+h, x:x+w]
                cropped_images.append(cropped_image)

    return edges, detected_image, cropped_images, detected_x, detected_y

def CreateDataSet(sDirektoriData, sKelas, NoKamera, FrameRate, recording_duration):
    sDirektoriKelas = os.path.join(sDirektoriData, sKelas)
    if not os.path.exists(sDirektoriKelas):
        os.makedirs(sDirektoriKelas)

    cap = cv2.VideoCapture(NoKamera)
    TimeStart = 0
    Recording = False
    recording_end_time = 0
    x, y = 0, 0  # Inisialisasi nilai x dan y

    while cap.isOpened():
        success, image = cap.read()

        if not success:
            print("Ignoring empty camera frame.")
            continue

        cv2.imshow('Original', image)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
        elif key == ord('t'):
            if not Recording:
                Recording = True
                TimeStart = time.time()
                recording_end_time = TimeStart + recording_duration  # Waktu akhir pencatatan

        if Recording:
            edges, detected_image, cropped_images, x, y = detect_contours(*preprocess_image(image))
            cv2.imshow('Filtered Edges', edges)
            cv2.imshow('Detected Contours', detected_image)

            TimeNow = time.time()
            if TimeNow - TimeStart > 1 / FrameRate and TimeNow < recording_end_time:
                for i, cropped_image in enumerate(cropped_images):
                    sfFile = os.path.join(sDirektoriKelas, f"cropped_{int(TimeNow * 1000)}_{x}_{y}_{i}.jpg")
                    cv2.imwrite(sfFile, cropped_image)  # Simpan gambar yang di-crop

                    cv2.imshow(f'Cropped Image {i}', cropped_image)

                TimeStart = TimeNow

            if TimeNow >= recording_end_time:
                print("Recording duration reached. Stopping the camera.")
                break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    sDirektoriData = "D:/PCV/project/dataset"
    sKelas = "4x0"
    NoKamera = 1  # Menggunakan kamera 1, sesuaikan jika menggunakan kamera lain
    FrameRate = 7
    recording_duration = 7  # Waktu pencatatan dalam detik

    # Inisialisasi variabel Recording
    Recording = False

    # Membuat dataset
    CreateDataSet(sDirektoriData, sKelas, NoKamera, FrameRate, recording_duration)

In [1]:
import os
import cv2
import ModulKlasifikasiCitraCNN as mCNN
from sklearn.model_selection import train_test_split

sDir = "D:/PCV/project/dataset"
files = os.listdir(sDir)  
for f in files:
  print(f)

0x0
1x0
1x1
2x0
2x1
2x2
3x0
3x1
3x2
3x3
4x0
4x1
4x2
4x3
4x4
5x0
5x1
5x2
5x3
5x4
5x5
6x0
6x1
6x2
6x3
6x4
6x5
6x6


In [2]:
LabelKelas = ['0x0', '1x0', '1x1', '2x0', '2x1', '2x2', '3x0', '3x1', '3x2', '3x3', '4x0', '4x1', '4x2', '4x3'
               ,'4x4', '5x0', '5x1', '5x2', '5x3', '5x4', '5x5', '6x0', '6x1', '6x2', '6x3', '6x4', '6x5', '6x6']


X, T = mCNN.LoadCitraTraining(sDir, LabelKelas)

cropped_1701368207401_243_142_0.jpg
cropped_1701368207560_243_142_0.jpg
cropped_1701368207728_243_143_0.jpg
cropped_1701368207877_243_142_0.jpg
cropped_1701368208065_241_136_0.jpg
cropped_1701368208221_241_139_0.jpg
cropped_1701368208397_242_142_0.jpg
cropped_1701368208553_239_144_0.jpg
cropped_1701368208728_236_145_0.jpg
cropped_1701368208896_237_142_0.jpg
cropped_1701368209060_237_143_0.jpg
cropped_1701368209221_237_148_0.jpg
cropped_1701368209378_237_149_0.jpg
cropped_1701368209563_236_151_0.jpg
cropped_1701368209711_235_151_0.jpg
cropped_1701368209885_232_153_0.jpg
cropped_1701368210071_232_155_0.jpg
cropped_1701368210219_228_156_0.jpg
cropped_1701368210397_222_161_0.jpg
cropped_1701368210587_217_169_0.jpg
cropped_1701368210741_217_167_0.jpg
cropped_1701368210901_217_166_0.jpg
cropped_1701368211065_220_167_0.jpg
cropped_1701368211216_221_167_0.jpg
cropped_1701368211391_217_162_0.jpg
cropped_1701368211545_209_161_0.jpg
cropped_1701368211699_199_159_0.jpg
cropped_1701368211863_196_15

In [3]:
JumlahEpoh = 10

# Tetapkan nama file untuk menyimpan model setelah training
FileBobot = "domino2.h5"

# Lakukan training model dengan memanggil fungsi TrainingCNN dari modul
ModelCNN, history = mCNN.TrainingCNN(JumlahEpoh, sDir, LabelKelas, FileBobot)

# Tampilkan summary dari model setelah proses training selesai
print(ModelCNN.summary())

cropped_1701368207401_243_142_0.jpg
cropped_1701368207560_243_142_0.jpg
cropped_1701368207728_243_143_0.jpg
cropped_1701368207877_243_142_0.jpg
cropped_1701368208065_241_136_0.jpg
cropped_1701368208221_241_139_0.jpg
cropped_1701368208397_242_142_0.jpg
cropped_1701368208553_239_144_0.jpg
cropped_1701368208728_236_145_0.jpg
cropped_1701368208896_237_142_0.jpg
cropped_1701368209060_237_143_0.jpg
cropped_1701368209221_237_148_0.jpg
cropped_1701368209378_237_149_0.jpg
cropped_1701368209563_236_151_0.jpg
cropped_1701368209711_235_151_0.jpg
cropped_1701368209885_232_153_0.jpg
cropped_1701368210071_232_155_0.jpg
cropped_1701368210219_228_156_0.jpg
cropped_1701368210397_222_161_0.jpg
cropped_1701368210587_217_169_0.jpg
cropped_1701368210741_217_167_0.jpg
cropped_1701368210901_217_166_0.jpg
cropped_1701368211065_220_167_0.jpg
cropped_1701368211216_221_167_0.jpg
cropped_1701368211391_217_162_0.jpg
cropped_1701368211545_209_161_0.jpg
cropped_1701368211699_199_159_0.jpg
cropped_1701368211863_196_15

In [10]:
from ModulKlasifikasiCitraCNN import LoadModel
import cv2
import numpy as np

def preprocess_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (3, 3), 0)
    
    # Filter Sobel untuk mendeteksi tepi
    sobel_x = cv2.Sobel(blurred, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(blurred, cv2.CV_64F, 0, 1, ksize=3)
    
    edge = cv2.bitwise_or(cv2.convertScaleAbs(sobel_x), cv2.convertScaleAbs(sobel_y))

    # edges = cv2.Canny(edge, 50, 150)
        
    _, edges = cv2.threshold(edge, 100, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    return edges, blurred

def main():
    cap = cv2.VideoCapture(1)
    # model_path = "path/to/your/trained/model.h5"  # Ganti dengan path model yang sudah dilatih
    model = LoadModel("domino.h5")  # Memuat model yang sudah dilatih

    while True:
        ret, frame = cap.read()

        live_frame = frame.copy()
        edges, preprocessed_frame = preprocess_image(frame)

        contours, _ = cv2.findContours(edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        for contour in contours:
            perimeter = cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, 0.04 * perimeter, True)
            
            if len(approx) == 4:
                x, y, w, h = cv2.boundingRect(contour)
                aspect_ratio = float(w) / h
                aspect_ratio_a = float(h) / w
                
                # Misalkan kartu memiliki perbandingan aspek mendekati 1
                # Sesuaikan dengan karakteristik kartu yang ingin dideteksi
                if 0.3 <= aspect_ratio <= 0.8 or 0.3 <= aspect_ratio_a <= 0.8 :
                    cv2.rectangle(live_frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

                    # Crop bagian gambar sesuai dengan kontur yang terdeteksi
                    cropped_image = edges[y:y+h, x:x+w]

                    cropped_image = cv2.cvtColor(cropped_image, cv2.COLOR_GRAY2BGR)

        X = []
        #                    v     dimasukin disini
        img = cv2.resize(cropped_image,(128,128))
        img = np.asarray(img)/255
        img = img.astype('float32')
        X.append(img)
        X = np.array(X)
        X = X.astype('float32')

        # Predict
        hs = model.predict(X,verbose = 0)
        n = np.max(np.where(hs== hs.max()))

                    # Prapemrosesan pada bagian yang di-crop
                    # processed_cropped_image = preprocess_cropped_image(cropped_image)

                    # Persiapkan gambar untuk diprediksi oleh model
                    # input_image = cv2.resize(processed_cropped_image, (ukuran_yang_diinginkan, ukuran_yang_diinginkan))  # Sesuaikan dengan ukuran yang diharapkan oleh model
                    # input_image = np.expand_dims(input_image, axis=0)  # Menambah dimensi batch jika diperlukan

                    # # Jika prapemrosesan menghasilkan gambar yang tepat untuk model, lakukan prediksi
                    # if processed_cropped_image is not None:
                    #     # Lakukan prediksi menggunakan model
                    #     predictions = model.predict(input_image)
                    #     predicted_class = np.argmax(predictions, axis=1)[0]
                    #     print(f"Prediksi: Kelas {predicted_class}")
                    #     # ... (logika lainnya sesuai kebutuhan)

        cv2.imshow("Deteksi Kartu Kotak", live_frame)
        cv2.imshow("Frame Hasil Pra-Pemrosesan - Tepi", edges)  # Tampilkan tepi Canny

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


ValueError: in user code:

    File "c:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\engine\training.py", line 2169, in predict_function  *
        return step_function(self, iterator)
    File "c:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\engine\training.py", line 2155, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\engine\training.py", line 2143, in run_step  **
        outputs = model.predict_step(data)
    File "c:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\engine\training.py", line 2111, in predict_step
        return self(x, training=False)
    File "c:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\USER

In [14]:
from ModulKlasifikasiCitraCNN import LoadModel
import cv2
import numpy as np

# Daftar label kelas
label_kelas = ['0x0', '1x0', '1x1', '2x0', '2x1', '2x2', '3x0', '3x1', '3x2', '3x3', '4x0', '4x1', '4x2', '4x3'
               ,'4x4', '5x0', '5x1', '5x2', '5x3', '5x4', '5x5', '6x0', '6x1', '6x2', '6x3', '6x4', '6x5', '6x6']

def preprocess_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (3, 3), 0)
    
    # Filter Sobel untuk mendeteksi tepi
    sobel_x = cv2.Sobel(blurred, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(blurred, cv2.CV_64F, 0, 1, ksize=3)
    
    edge = cv2.bitwise_or(cv2.convertScaleAbs(sobel_x), cv2.convertScaleAbs(sobel_y))

    _, edges = cv2.threshold(edge, 100, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    return edges, blurred


def main():
    cap = cv2.VideoCapture(1)
    model_path = "domino2.h5"  # Path model yang sudah dilatih
    model = LoadModel(model_path)  # Memuat model yang sudah dilatih

    while True:
        ret, frame = cap.read()
        height, width, _ = frame.shape

        live_frame = frame.copy()
        edges, preprocessed_frame = preprocess_image(frame)

            # Membagi bingkai menjadi tiga bagian horizontal
        cv2.line(live_frame, (0, height // 3), (width, height // 3), (0, 255, 0), 2)
        cv2.line(live_frame, (0, height // 3 * 2), (width, height // 3 * 2), (0, 255, 0), 2)
    

    # Menambah garis vertikal di tengah
        center_line_length = height // 3
        center_line_x = width // 2
        cv2.line(live_frame, (center_line_x, width // 2), (center_line_x, height//3), (0, 255, 0), 2)


        # player = live_frame[0:height//3]
        # computer = live_frame[height // 3*2:height, :]
        # player_play = live_frame[height//3 : height//3*2, : width // 2]
        # computer_play= live_frame[height//3 : height // 3*2,  width // 2 :]


    # Menampilkan bingkai dengan garis pembagi
        # cv2.imshow('frame', frame)
        # cv2.imshow('Bagian 1', bagian_1)
        # cv2.imshow('Bagian 2', bagian_2)
        # cv2.imshow('Bagian 3', bagian_3)
        # cv2.imshow('Bagian 4', bagian_4)

        contours, _ = cv2.findContours(edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        for contour in contours:
            perimeter = cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, 0.04 * perimeter, True)
            
            if len(approx) == 4:
                x, y, w, h = cv2.boundingRect(contour)
                aspect_ratio = float(w) / h
                aspect_ratio_a = float(h) / w
                
                # Misalkan kartu memiliki perbandingan aspek mendekati 1
                # Sesuaikan dengan karakteristik kartu yang ingin dideteksi
                if 0.3 <= aspect_ratio <= 0.8 or 0.3 <= aspect_ratio_a <= 0.8 :
                    cv2.rectangle(live_frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

                    # Crop bagian gambar sesuai dengan kontur yang terdeteksi
                    cropped_image = edges[y:y+h, x:x+w]

                    cropped_image = cv2.cvtColor(cropped_image, cv2.COLOR_GRAY2BGR)

                    X = []
                    # Persiapan input untuk prediksi model
                    img = cv2.resize(cropped_image, (128, 128))
                    img = np.asarray(img)/255
                    img = img.astype('float32')
                    X.append(img)
                    X = np.array(X)
                    X = X.astype('float32')

                    # Lakukan prediksi menggunakan model
                    hs = model.predict(X, verbose=0)
                    predicted_class_idx = np.argmax(hs)
                    predicted_class = label_kelas[predicted_class_idx]

                    # Menuliskan hasil prediksi pada frame
                    cv2.putText(live_frame, f"{predicted_class}", (x, y - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2, cv2.LINE_AA)
                    

        cv2.imshow("Deteksi Kartu Kotak", live_frame)
        cv2.imshow("Frame Hasil Pra-Pemrosesan - Tepi", edges)  # Tampilkan tepi Canny

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


In [10]:
import random

class Domino:
    def __init__(self):
        self.domino_tiles = []
        self.player_hand = []
        self.computer_hand = []

    def create_tiles(self):
        for i in range(7):
            for j in range(i, 7):
                self.domino_tiles.append((i, j))

    def shuffle_tiles(self):
        random.shuffle(self.domino_tiles)

    def deal_tiles(self):
        self.player_hand = self.domino_tiles[:7]
        self.computer_hand = self.domino_tiles[7:14]
        self.domino_tiles = self.domino_tiles[14:]

    def display_hand(self, hand):
        print("Domino Tiles:")
        for tile in hand:
            print(tile)

    def start_game(self):
        self.create_tiles()
        self.shuffle_tiles()
        self.deal_tiles()

        while True:
            print("\nPlayer's Hand:")
            self.display_hand(self.player_hand)
            
            print("\nComputer's Hand:")
            self.display_hand(self.computer_hand)

            if len(self.player_hand) == 0:
                print("Congratulations! Player wins!")
                break
            elif len(self.computer_hand) == 0:
                print("Computer wins! Better luck next time.")
                break

            player_choice = int(input("\nEnter the index of tile to play: "))
            while player_choice < 0 or player_choice >= len(self.player_hand):
                player_choice = int(input("Invalid input! Enter the index of tile to play: "))

            tile_to_play = self.player_hand.pop(player_choice)
            print(f"Player plays: {tile_to_play}")

            for i, tile in enumerate(self.computer_hand):
                if tile[0] == tile_to_play[0] or tile[1] == tile_to_play[0] or tile[0] == tile_to_play[1] or tile[1] == tile_to_play[1]:
                    print(f"Computer plays: {tile}")
                    self.computer_hand.pop(i)
                    break
            else:
                print("Computer draws a tile.")
                self.computer_hand.append(self.domino_tiles.pop(0))

domino_game = Domino()
domino_game.start_game()



Player's Hand:
Domino Tiles:
(3, 4)
(4, 4)
(0, 2)
(0, 6)
(0, 4)
(1, 5)
(1, 6)

Computer's Hand:
Domino Tiles:
(4, 5)
(1, 4)
(2, 4)
(3, 5)
(5, 5)
(5, 6)
(1, 2)


ValueError: invalid literal for int() with base 10: ''

In [14]:
import cv2


# Inisialisasi kamera
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    height, width, _ = frame.shape

    # Membagi bingkai menjadi tiga bagian horizontal
    cv2.line(frame, (0, height // 3), (width, height // 3), (0, 255, 0), 2)
    cv2.line(frame, (0, height // 3 * 2), (width, height // 3 * 2), (0, 255, 0), 2)
    

    # Menambah garis vertikal di tengah
    center_line_length = height // 3
    center_line_x = width // 2
    cv2.line(frame, (center_line_x, width // 2), (center_line_x, height//3), (0, 255, 0), 2)


    bagian_1 = frame[0:height//3]
    bagian_2 = frame[height // 3*2:height, :]
    bagian_3 = frame[height//3 : height//3*2, : width // 2]
    bagian_4 = frame[height//3 : height // 3*2,  width // 2 :]


    # Menampilkan bingkai dengan garis pembagi
    cv2.imshow('frame', frame)
    cv2.imshow('Bagian 1', bagian_1)
    cv2.imshow('Bagian 2', bagian_2)
    cv2.imshow('Bagian 3', bagian_3)
    cv2.imshow('Bagian 4', bagian_4)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [7]:
label_kelas = ['0x0', '1x0', '1x1', '2x0', '2x1', '2x2', '3x0', '3x1', '3x2', '3x3', '4x0', '4x1', '4x2', '4x3',
               '4x4', '5x0', '5x1', '5x2', '5x3', '5x4', '5x5', '6x0', '6x1', '6x2', '6x3', '6x4', '6x5', '6x6']

def jumlahkan_angka_list(label_kelas):
    hasil_jumlah_list = []
    for label in label_kelas:
        angka = int(label.split('x')[0]) + int(label.split('x')[1])
        hasil_jumlah_list.append(angka)
    return hasil_jumlah_list

hasil_jumlah_list = jumlahkan_angka_list(label_kelas)
print("Hasil penjumlahan angka di setiap list label kelas:", hasil_jumlah_list)


Hasil penjumlahan angka di setiap list label kelas: [0, 1, 2, 2, 3, 4, 3, 4, 5, 6, 4, 5, 6, 7, 8, 5, 6, 7, 8, 9, 10, 6, 7, 8, 9, 10, 11, 12]


In [7]:
from ModulKlasifikasiCitraCNN import LoadModel
import cv2
import numpy as np
import time

label_kelas = ['0x0', '1x0', '1x1', '2x0', '2x1', '2x2', '3x0', '3x1', '3x2', '3x3', '4x0', '4x1', '4x2', '4x3',
               '4x4', '5x0', '5x1', '5x2', '5x3', '5x4', '5x5', '6x0', '6x1', '6x2', '6x3', '6x4', '6x5', '6x6']

def preprocess_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (3, 3), 0)
    
    # Filter Sobel untuk mendeteksi tepi
    sobel_x = cv2.Sobel(blurred, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(blurred, cv2.CV_64F, 0, 1, ksize=3)
    
    edge = cv2.bitwise_or(cv2.convertScaleAbs(sobel_x), cv2.convertScaleAbs(sobel_y))

    _, edges = cv2.threshold(edge, 100, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    return edges, blurred

def jumlahkan_angka_list(label_kelas):
    hasil_jumlah_list = {}
    for label in label_kelas:
        angka = int(label.split('x')[0]) + int(label.split('x')[1])
        hasil_jumlah_list[label] = angka
    return hasil_jumlah_list

def find_valid_card_for_computer(computer_card, player_play_card):
    valid_cards = []

    for card in computer_card:
        for played_card in player_play_card:
            card_values = card.split('x')
            played_card_values = played_card.split('x')

    
            if card_values[0] == played_card_values[0] or card_values[1] == played_card_values[1] or card_values[0] == played_card_values[1] :
                valid_cards.append(card)
                break  
    return valid_cards

def check_validity(card_entered):
    card1 = list(map(int, card_entered[0].split('x')))
    card2 = list(map(int, card_entered[1].split('x')))
    
    return any(item in card1 for item in card2)

def is_empty(obj):
    if not obj: 
        return True
    else:  
        return False

def is_not_empty(obj):
    return not is_empty(obj)
    
def show_player_play_window(kartu_terbesar_player):
    player_window = np.ones((300, 500, 3), dtype=np.uint8) * 255 
    cv2.putText(player_window, f"Player's Turn: Play {kartu_terbesar_player}", (50, 150), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
    return player_window

def show_player_play(kartu_player):
    player_window_play = np.ones((300, 500, 3), dtype=np.uint8) * 255  
    cv2.putText(player_window_play, "Masukkan Kartu Anda", (50, 150), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
    return player_window_play

def show_computer_play_window(kartu_terbesar_komputer):
    computer_window = np.ones((300, 500, 3), dtype=np.uint8) * 255 
    cv2.putText(computer_window, f"Computer's Turn: Play {kartu_terbesar_komputer}", (10, 150),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), 2, cv2.LINE_AA)
    return computer_window

def show_computer_play(kartu_komputer):
    computer_play_window = np.ones((300, 500, 3), dtype=np.uint8) * 255  
    cv2.putText(computer_play_window, f"Computer's Turn: Play {kartu_komputer}", (10, 150),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), 2, cv2.LINE_AA)
    return computer_play_window

def pass_play(kartu):
    pass_window = np.ones((300, 500, 3), dtype=np.uint8) * 255  
    cv2.putText(pass_window, "PASS PRESS R", (10, 150),
                cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 0), 2, cv2.LINE_AA)
    return pass_window


def main():
    cap = cv2.VideoCapture(1)  
    model_path = "domino2.h5"
    model = LoadModel(model_path)

    global player_play_window_open, player_play_open
    global computer_play_window_open, computer_play_open

    player_play_window_open = True
    computer_play_window_open = True
    computer_play_open = True
    player_play_open = True
    
    game_start = False

    hasil_jumlah_list = jumlahkan_angka_list(label_kelas)

    while True:
        ret, frame = cap.read()
        height, width, _ = frame.shape
        
        card_entered = []
        player_play_card = []
        computer_play_card = []
        player_card = []
        computer_card = []

        live_frame = frame.copy()
        edges, preprocessed_frame = preprocess_image(frame)


        cv2.line(live_frame, (0, height // 3), (width, height // 3), (0, 255, 0), 2)
        cv2.line(live_frame, (0, height // 3 * 2), (width, height // 3 * 2), (0, 255, 0), 2)

        cv2.line(live_frame, (width // 2, width // 2), (width // 2, height//3), (0, 255, 0), 2)


        contours, _ = cv2.findContours(edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        for contour in contours:
            perimeter = cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, 0.04 * perimeter, True)
            
            if len(approx) == 4:
                x, y, w, h = cv2.boundingRect(contour)
                aspect_ratio = w
                aspect_ratio_a = h
                
                if 0 <= aspect_ratio <= 80 and 80 <= aspect_ratio_a <= 170:
                    cv2.rectangle(live_frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

                    cropped_image = edges[y:y+h, x:x+w]
                    cropped_image = cv2.cvtColor(cropped_image, cv2.COLOR_GRAY2BGR)

                    X = []
                    img = cv2.resize(cropped_image, (128, 128))
                    img = np.asarray(img)/255
                    img = img.astype('float32')
                    X.append(img)
                    X = np.array(X)
                    X = X.astype('float32')

                    hs = model.predict(X, verbose=0)
                    predicted_class_idx = np.argmax(hs)
                    predicted_class = label_kelas[predicted_class_idx]

                    if y < height // 3:
                        player_card.append(predicted_class)
                        cv2.putText(live_frame, f"player: {predicted_class}", (x -30 , y - 5),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
                    elif y > height // 3 * 2:
                        computer_card.append(predicted_class)
                        cv2.putText(live_frame, f"Komputer: {predicted_class}", (x -30, y - 5),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
                    
                    elif height // 3 < y < height // 3 * 2 and x < width // 2:
                        player_play_card.append(predicted_class)
                        cv2.putText(live_frame, f"player_play: {predicted_class}", (x -30, y - 5),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
                        
                    else :
                        computer_play_card.append(predicted_class)
                        cv2.putText(live_frame, f"computer_play: {predicted_class}", (x -30, y - 5),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
                        
        if game_start:
            print("computer play", computer_play_card)
            print("player play", player_play_card)
            card_entered = player_play_card + computer_play_card
            # print(card_entered)
            valid_cards = find_valid_card_for_computer(computer_card, player_play_card)
            text_computer = (100, height // 2)
            text_player = (50, height//2)
            if len(player_card) == 0 and len(computer_card) == 0:
                print("kartu tidak terdeteksi")
                continue
            if len(player_card) == 0:
                cv2.putText(live_frame, "PLAYER WIN!!!", text_player, cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 0, 255), 2, cv2.LINE_AA)
            elif len(computer_card) == 0:
                cv2.putText(live_frame, "COMPUTER WIN!!!", text_player, cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 0, 255), 2, cv2.LINE_AA)

            contours, _ = cv2.findContours(edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            most_common_label = max(player_card + computer_card + player_play_card + computer_play_card, key=lambda x: hasil_jumlah_list.get(x, 0))

            if most_common_label in player_card or most_common_label in player_play_card:
                computer_play_window_open = False
                player_play_window_open
                if player_play_window_open:
                    player_window = show_player_play_window(most_common_label)
                    cv2.imshow("Player Play", player_window)
                    if cv2.waitKey(1) == ord(' '):
                        cv2.destroyWindow("Player Play")
                        player_play_window_open = False
                        if is_empty(player_play_card):
                            cv2.putText(live_frame, "masukkan kartu player", text_player, cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
                if is_not_empty(player_play_card):
                        print("xxxx")
                        computer_play_open
                        if computer_play_open:
                            computer_valid = show_computer_play(valid_cards)
                            cv2.imshow("Computer Play2", computer_valid)
                            if is_empty(valid_cards):
                                cv2.destroyWindow("Computer Play2")
                                computer_play_open = False
                                is_empty(player_play_card)
                                pass_computer = pass_play(valid_cards)
                                cv2.imshow("leat", pass_computer)
                            if cv2.waitKey(1) == ord(' '):
                                cv2.destroyWindow("Computer Play2")
                                computer_play_open = False
                                if is_empty(computer_play_card):
                                    cv2.putText(live_frame, "Masukkan Kartu Komputer", text_player, cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
                        if is_not_empty(computer_play_card):
                                print("guss")
                                is_valid = check_validity(card_entered)
                                if is_valid:
                                    print("----")
                                    computer_play_open
                                    player_play_window_open 
                                    player_play_open
                                    computer_play_window_open
                                    cv2.putText(live_frame, "lanjut teka R", text_player, cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
                                else :
                                    print("salah")
                                    cv2.putText(live_frame, "SALAH", text_player, cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

            elif most_common_label in computer_card or most_common_label in computer_play_card:
                player_play_window_open = False
                computer_play_window_open
                if computer_play_window_open:
                    computer_window = show_computer_play_window(most_common_label)
                    cv2.imshow("Computer Play", computer_window)
                    if cv2.waitKey(1) == ord(' '):
                        cv2.destroyWindow("Computer Play")
                        computer_play_window_open = False
                        if is_empty(computer_play_card):
                            cv2.putText(live_frame, "masukkan kartu computer", text_computer, cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
                if is_not_empty(computer_play_card):
                    print("masuk")
                    player_play_open
                    if player_play_open:
                        player_main = show_player_play(valid_cards)
                        cv2.imshow("Player Turn", player_main)
                        if cv2.waitKey(1) == ord(' '):
                            cv2.destroyWindow("Player Turn")
                            player_play_open = False
                            if is_empty(player_play_card):
                                cv2.putText(live_frame, "Masukkan", text_player, cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
                        if cv2.waitKey(1) == ord('p'):
                            cv2.destroyWindow("Player Turn")
                            player_play_open = False
                            pass_player = pass_play(valid_cards)
                            cv2.imshow("lewat", pass_player)
                            is_empty(player_play_card)
                            cv2.putText(live_frame, "lanjut", text_player, cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
                    if is_not_empty(player_play_card):
                        print("guss")
                        is_valid = check_validity(card_entered)
                        if is_valid:
                            print("hgggg")
                            player_play_open 
                            computer_play_window_open
                            player_play_window_open 
                            computer_play_open
                            cv2.putText(live_frame, "LANJUT PRESS R", text_player, cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
                        else :
                            print("salah")
                            cv2.putText(live_frame, "salahsalah", text_player, cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)


        cv2.imshow("Deteksi Kartu Kotak", live_frame)
        cv2.imshow("Frame Hasil Pra-Pemrosesan - Tepi", edges) 

        if cv2.waitKey(1) == ord('t') and not game_start:
            game_start = True
            print("mulai")

        if cv2.waitKey(1) == ord('q'):
            break

        if cv2.waitKey(1) == ord('r'):  
            cv2.destroyAllWindows()  
            computer_play_window_open = True
            player_play_window_open = True
            computer_play_open = True
            player_play_open = True
            game_start = True

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

In [7]:
q

False


In [38]:
def check_card_validity(card_entered):
    # Mengecek kesamaan antara dua kartu yang disambungkan
    

card_entered = ['6x6', '6x6']

is_valid = check_card_validity(card_entered)
print(is_valid)

False


In [2]:
def check_match(card_entered):
    # Pisahkan angka-angka dalam kartu
    card1 = list(map(int, card_entered[0].split('x')))
    card2 = list(map(int, card_entered[1].split('x')))
    
    # Periksa kesamaan antara dua list angka
    return any(item in card1 for item in card2)

# Contoh penggunaan
card_entered = ['6x1', '4x1']
hasil = check_match(card_entered)
print(hasil)


True
